In [1]:
# Load the gams extension
%load_ext gams_magic

# Job shop scheduling

A job consists of making three parts.  Part 1 has three operations that have to be
done in order. Part 2 has two operations that can be done in any order. Part 3
has three operations, both operation 1 and 2 must be done before operation 3.

There are two machines. Each operation has to be done on a particular machine.

### Minimize total completion time.  

Key issues to deal with are:
- A machine cannot process two tasks at once.
- Two operations on the same part cannot be processed concurrently.

Tasks are defined by pairs (part,operation) that have to get done!
Processing times (and their machines) are given below:

In [2]:
%%gams
set part /p1*p3/;
set oper /o1*o3/;
set machine /m1*m2/;

parameter procdata(part,oper,machine) /
p1.o1.m1      3
p1.o2.m1      5
p1.o3.m2      6
p2.o1.m1      8
p2.o2.m2      4
p3.o1.m2      9
p3.o2.m1      2
p3.o3.m2      7
/;

In [3]:
%%gams
set orden(part,oper,machine,part,oper,machine) /
p1.o1.m1.p1.o2.m1
p1.o1.m1.p2.o1.m1 
p1.o1.m1.p3.o2.m1
p1.o2.m1.p2.o1.m1
p1.o2.m1.p3.o2.m1
p2.o1.m1.p3.o2.m1
p3.o1.m2.p3.o2.m1
p1.o3.m2.p2.o2.m2
p1.o3.m2.p3.o1.m2
p1.o3.m2.p3.o3.m2
p2.o2.m2.p3.o1.m2
p2.o2.m2.p3.o3.m2
p3.o1.m2.p3.o3.m2
p2.o1.m1.p2.o2.m2
p1.o1.m1.p1.o3.m2
p1.o2.m1.p1.o3.m2
p3.o2.m2.p3.o3.m2
/;

alias(oper,o,oo)
alias(part,p,pp)
alias(machine,m,mm)

variable comp(part);
positive variable start(p,o,m);
variable T;     

equation Tdef;
Tdef(part)..
T =g= comp(part);

equation compDef;
compDef(part,oper,machine)..
comp(part) =g= start(part,oper,machine) + procdata(part,oper,machine);

equation orderEq; 
orderEq( orden(p,o,m,pp,oo,mm) )..
start(p,o,m) + procdata(p,o,m) =l= start(pp,oo,mm);

model jobshop /all/;
solve jobshop using MIP min T;
display start.L, comp.L, T.l;



,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),36.0,38,22,MIP,CPLEX,0.016


In [4]:
# start time of task, completion time of part
%gams_pull -d start comp
display(start,comp)

,part,oper,machine,level,marginal,lower,upper,scale
0,p1,o1,m1,0.0,1.000000e+00,0.0,inf,1.0
1,p1,o1,m2,0.0,4.940656e-324,0.0,inf,1.0
2,p1,o2,m1,3.0,0.000000e+00,0.0,inf,1.0
3,p1,o2,m2,0.0,4.940656e-324,0.0,inf,1.0
4,p1,o3,m1,0.0,4.940656e-324,0.0,inf,1.0
5,p1,o3,m2,8.0,0.000000e+00,0.0,inf,1.0
6,p2,o1,m1,8.0,0.000000e+00,0.0,inf,1.0
7,p2,o1,m2,0.0,4.940656e-324,0.0,inf,1.0
8,p2,o2,m1,0.0,4.940656e-324,0.0,inf,1.0
9,p2,o2,m2,16.0,0.000000e+00,0.0,inf,1.0


,part,level,marginal,lower,upper,scale
0,p1,36.0,0.0,-inf,inf,1.0
1,p2,36.0,0.0,-inf,inf,1.0
2,p3,36.0,0.0,-inf,inf,1.0
